This notebook exists so that we can try and clean up the data missing from UK biobank... preliminary work suggests that we can download anythign that is listed in the bulk files.

In [ ]:
import os

import zipfile

from itertools import compress

Load and parse all .bulk files, looking only at their union, and subset for those referring to CMR data.

In [ ]:
#load bulk files, this will produce a list of strings
bulkFiles = ['/workspace/storage/restricted-biobank/releases/REVISION_May2019/ID-27289/ukb27289.bulk','/workspace/storage/restricted-biobank/releases/REVISION_May2019/ID-29801/ukb29801.bulk']

bulk = []

for b in bulkFiles:
    with open(b,'r') as f:
        bulk+=f.readlines()

#uniqueify
bulk = list(set(bulk))

print('total of ' + str(len(bulk)) + ' files listed in bulk')

In [ ]:
#define dictionaries for mapping between human-readable names of zipfiles, and the UKB codes

code2name = {'20207_2_0':'_scout',
             '20208_2_0':'_longaxis',
             '20209_2_0':'_shortaxis',
             '20210_2_0':'_aorticdistensibility',
             '20211_2_0':'_cinetagging',
             '20212_2_0':'_lvot',
             '20213_2_0':'_bloodflow',
             '20214_2_0':'_experimentalshmollisequence',
            }

#FIXME! THERE ARE 2 DIFFERENT SPELLINGS OF '_aorticdistensibility'/'_aorticdistensibilty'

name2code = {v:k for k,v in code2name.items()}

#loop over bulk list, keeping elements which have been marked as interesting
bulk = [f for f in bulk if f[8:-1] in code2name.keys()]

print('subsetted for only cmr, there are ' + str(len(bulk)) + ' files.')


Convert the resulting list into names of zipfiles. 

Check for the existence of those zipfiles at the appropriate location, and subset.

In [ ]:
def bulk2zip(bulk):
    
    feid = bulk[:7]
    
    code = bulk[8:-1]
    
    extname = code2name[code]
    
    return  feid + extname + '.zip'


def zip2path(zipName):
    
    feid = zipName[:7]
    
    path = os.path.join('./data/imaging_by_participant',feid[:2] + 'xxxxx',feid,zipName)
    
    return path

In [ ]:
zipNames = [bulk2zip(f) for f in bulk]

zipPaths = [zip2path(z) for z in zipNames]

notThere = [not os.path.isfile(f) for f in zipPaths]

print('of ' + str(len(bulk)) + ' cmr files listed in bulk, there are ' + str(sum(notThere)) + ' files missing (' + str(100*sum(notThere)/len(bulk)) + '%).')

Split into batches of 500 and use ukbfetch, checking which examples of each batch have failed to download

Check the integrity of the zipfiles (!), and move valid ones into a directory structure consistent with that already used so it can be merged.